# Tutorial 5. Layout

:::{note}
:icon: false

#### Sizing and responsiveness: fixed vs stretch

The most important layout skill in Panel is understanding sizing behavior.  
Whether you use classic Panel components or `panel_material_ui`, the same core model applies:

- fixed size (`width` / `height`)
- responsive stretch (`sizing_mode`)
- constraints (`min_*`, `max_*`)

In this tutorial you will practice these rules and apply them in both APIs.

## Learning goals

By the end, you should be able to:

1. Predict when a component keeps a fixed size vs stretches.
2. Use `min_width` / `max_width` to keep responsive layouts usable.
3. Apply the same sizing ideas to classic Panel and `panel_material_ui`.

:::

---

In [ ]:
import panel_material_ui as pmui
import panel as pn

pn.extension("tabulator")

## 1) The sizing toolbox

The four parameters to remember:

`width` / `height`
: Set fixed dimensions.

`sizing_mode`
: Controls responsiveness: `fixed`, `stretch_width`, `stretch_height`, `stretch_both`, `scale_width`, `scale_height`, `scale_both`.

`min_width` / `min_height`
: Lower bounds for responsive layouts.

`max_width` / `max_height`
: Upper bounds for responsive layouts.

---

## 2) Inherent size vs fixed size

Some content (like Markdown text) has an inherent size and wraps as needed.

In [ ]:
lorem_ipsum = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum."
pn.pane.Markdown(lorem_ipsum)

By restricting the width, we can force it to rewrap and it will have a different inherent height.

In [ ]:
pn.pane.Markdown(lorem_ipsum, width=300)

Explicitly setting both width and height will force the resulting display to scroll to ensure that it is not cut off:

In [ ]:
pn.pane.Markdown(lorem_ipsum, width=300, height=100)

## 3) Responsive sizing with `sizing_mode`

To understand stretch behavior, place components inside a fixed-size container:


In [ ]:
frame = {"width": 420, "height": 300, "styles": {"border": "1px solid #aaa"}, "margin": 10}

width_responsive = pn.Spacer(
    styles={"background": "indianred"},
    sizing_mode="stretch_width",
    height=80,
)

pn.Column(width_responsive, **frame)


In [ ]:
height_responsive = pn.Spacer(
    styles={"background": "seagreen"},
    sizing_mode="stretch_height",
    width=120,
)

pn.Column(height_responsive, **frame)

In [ ]:
both_responsive = pn.Spacer(
    styles={"background": "royalblue"},
    sizing_mode="stretch_both",
)

pn.Column(both_responsive, **frame)


### Mini Exercise 1

Arrange a Markdown pane and a Bokeh figure in a row so that:

- both fill available width
- the text pane stays between `200` and `500` px
- the row stays `500` px tall

In [ ]:
import numpy as np
from bokeh.plotting import figure

xs = np.linspace(0, 10)
ys = np.sin(xs)

md = pn.pane.Markdown(lorem_ipsum)   # add sizing options
fig = figure()                       # add sizing options
fig.line(xs, ys)

pn.Row(fig, md, height=500, sizing_mode="stretch_width")

:::{note} Solution
:class: dropdown

```python
md = pn.pane.Markdown(
    lorem_ipsum,
    sizing_mode="stretch_width",
    min_width=200,
    max_width=500,
)
fig = figure(sizing_mode="stretch_both")
```
:::

## 5) True responsive layouts with `FlexBox`

Stretching one component is not the same as responsive page reflow.  
For reflow behavior, use `pn.FlexBox`.

In [ ]:
import random

pn.FlexBox(
    *(
        pn.Spacer(
            height=100,
            width=random.randint(1, 4) * 90,
            styles={"background": "indianred"},
            margin=5,
        )
        for _ in range(8)
    )
)

`FlexBox` is based on [CSS Flexbox](https://css-tricks.com/snippets/css/a-guide-to-flexbox/) and supports many of the same options, such as setting `flex_direction`, `flex-wrap`, `align_items` and `align_content`.

### Controlling proportions with `flex`

In [ ]:
left = pn.Spacer(height=140, styles={"background": "#c62828", "flex": "1 1 0"})
center = pn.Spacer(height=140, styles={"background": "#2e7d32", "flex": "3 1 0"})
right = pn.Spacer(height=140, styles={"background": "#1565c0", "flex": "1 1 0"})

pn.FlexBox(left, center, right, sizing_mode="stretch_width")

### Mini Exercise 2

Using `pn.FlexBox`, place two plots and one Markdown pane so each plot is about 2x as wide as the text pane, and center the text vertically.

In [ ]:
sin_fig = figure()
sin_fig.line(xs, ys)
cos_fig = figure()
cos_fig.line(xs, np.cos(xs))
text = pn.pane.Markdown(lorem_ipsum)
pn.Row(sin_fig, text, cos_fig, sizing_mode="stretch_width")

:::{note} Solution
:class: dropdown

```python
sin_fig = figure(height=360, sizing_mode="stretch_width", styles={"flex": "2 1 0"})
sin_fig.line(xs, ys)
cos_fig = figure(height=360, sizing_mode="stretch_width", styles={"flex": "2 1 0"})
cos_fig.line(xs, np.cos(xs))
text = pn.pane.Markdown(lorem_ipsum, styles={"flex": "1 1 0", "align-self": "center"})
pn.FlexBox(sin_fig, text, cos_fig, sizing_mode="stretch_width")
```
:::

## 7) Section Recap Exercise

Build a responsive dashboard from the components declared below.

- a fixed-width filter panel (`width=340`)
- a main content area that stretches

Some additional constraints:

1. Keep the app usable between 900px and 1600px wide.
2. Use `min_width` and `max_width` at least once.
3. Use `stretch_both` in at least one main content component.

Starter:

In [ ]:
import holoviews as hv
import hvplot.pandas
import pandas as pd

df = pn.rx(pd.read_parquet('./windturbines.parq'))

manufacturers = pmui.MultiChoice(
    options=df.t_manu.unique().rx.pipe(list),
    label="Manufacturer",
)
year = pmui.IntRangeSlider(
    start=df.p_year.min().rx.pipe(int),
    end=df.p_year.max().rx.pipe(int),
    value=(2000, 2004),
    label="Year",
)
columns = ["p_name", "t_state", "t_county", "p_year", "t_manu", "p_cap"]

filtered = df[columns][
    df.t_manu.isin(manufacturers.rx.where(manufacturers, df.t_manu.unique()))
    & df.p_year.between(*year.rx())
]

year_hist = pn.pane.HoloViews(filtered.hvplot.hist(y='p_year', responsive=True, height=312))

cap_hist = pn.pane.HoloViews(filtered.hvplot.hist(y='p_cap', responsive=True, height=312))

count = pn.indicators.Number(name="Turbine Count", value=filtered.rx.len(), format="{value:,d}")

total_cap = pn.indicators.Number(name="Total Capacity", value=filtered.p_cap.mean(), format="{value:.2f} kW")

:::{note} Solution
:class: dropdown

```
filters = pmui.Card(
    manufacturers,
    year,
    title="Filters",
    width=340,
    margin=(0, 10, 0, 0)
)

summary = pmui.Card(
    pn.Row(count, total_cap),
    title="Summary",
    sizing_mode="stretch_width",
    margin=(0, 0, 5, 0),
)

plots = pmui.Card(
    pn.Row(year_hist, cap_hist, sizing_mode="stretch_width"),
    sizing_mode="stretch_width",
    min_height=300
)

main = pn.Column(
    summary,
    plots,
    sizing_mode="stretch_both",
)

pn.Row(filters, main, sizing_mode="stretch_both", min_height=650)
```
:::

## Recap

You now have the core layout rules needed for real apps:

- choose fixed sizing (`width`/`height`) vs responsive sizing (`sizing_mode`) intentionally
- use `min_*` and `max_*` constraints to keep layouts usable
- apply the same sizing model across classic Panel and `panel_material_ui` components